In [1]:
import os
import glob
from lxml import etree
import pandas as pd
import re
import requests

In [2]:
sys.path.append(os.path.abspath("./../../../code/"))

In [3]:
from fachreferats_functions import clean_data, check_duplicates, extract_data, work_with_zotero


# Get Data From Zotero

Bibliografie: https://www.zotero.org/groups/2360892/bibliografie_zur_digitalen_romanistik

In [4]:
data = work_with_zotero.download_zotero_bibliography_json("2360892", item_type = "book || bookSection")


https://api.zotero.org/groups/2360892/items?format=json&limit=100&itemType=book || bookSection


In [5]:
data[0]["data"].keys()

dict_keys(['key', 'version', 'itemType', 'title', 'creators', 'abstractNote', 'series', 'seriesNumber', 'volume', 'numberOfVolumes', 'edition', 'place', 'publisher', 'date', 'numPages', 'language', 'ISBN', 'shortTitle', 'url', 'accessDate', 'archive', 'archiveLocation', 'libraryCatalog', 'callNumber', 'rights', 'extra', 'tags', 'collections', 'relations', 'dateAdded', 'dateModified'])

In [6]:
dataframe = work_with_zotero.convert_zotero_json_to_zotero_csv(data)

In [7]:
dataframe

,key,itemType,title,bookTitle,firstName,lastName,publisher,place,date,language,numPages,ISBN,url
0,9QXAD3VR,book,Gattungspoetik in quantitativer Sicht. Das Wer...,NaN,Hanno,Ehrlicher,Digital Humanities Cooperation,,2020,deutsch,1-29,,https://www.digitalhumanitiescooperation.de/wp...
1,WWI35ZEX,book,Theorien von Autorschaft und Stil in Bewegung:...,NaN,Nanette,Rißler-Pipka,,,2019,German,,978-3-95477-099-1,http://romanischestudien.de/index.php/rst/issu...
2,EH82H9XH,book,"Schäferei, Computer, Internet. Digital Humanit...",NaN,Reinhard,Krüger,V&R unipress,Göttingen/Bonn,2016,,,,
3,QEI4T8ZU,bookSection,Das aleatorische Netz des Wissens: Die Traditi...,Europäische Verlage und romanische Gegenwartsl...,Reinhard,Krüger,Narr,Tübingen,2003,de,NaN,,
4,LYJ9N5F8,book,Multilingual corpora and multilingual corpus a...,NaN,Thomas,Schmidt,Benjamins,"Amsterdam, Philadelphia",2012,en,,,https://benjamins.com/catalog/hsm.14
5,3ZT9YJ33,bookSection,"Edición de textos, hipertextos y lexicografía",Actes du XXVe Congrès International de Linguis...,Harald,Völker,de Gruyter,Innsbruck,2010,,NaN,,https://doi.org/10.5167/uzh-45415
6,JTIY67LY,bookSection,A survey of spoken language corpora in Romance,Romanistische Korpuslinguistik. Korpora und ge...,Claus D.,Pusch,Narr,Tübingen,2003,de,NaN,,https://home.uni-leipzig.de/burr/CorpusLing/pd...
7,M6ED3TMH,bookSection,"Corneille, Molière et les autres. Stilometrisc...",Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,NaN,,http://web.fu-berlin.de/phin/beiheft7/b7t08.pdf
8,4YGT9GT7,book,Literaturwissenschaft im digitalen Medienwandel,NaN,Christof,Schöch,Philologie im Netz,,2014,de,,,http://web.fu-berlin.de/phin/beiheft7/b7i.htm
9,LZZQJCY7,bookSection,El estudio de revistas culturales en la era de...,Tramas impresas. Publicaciones periódicas arge...,Hanno,Ehrlicher,Universidad Nacional de La Plata: Facultad de ...,,2015,es,NaN,,


# Transform tables

In [8]:
dataframe = clean_data.clean_data(dataframe)

The column  Titel  is mandatory but missing in the table. We are going to try to find another one in the table that contains this information, but perhaps this creates errors!
Replacing  title  with  Titel
The column  Vorname_Autor  is mandatory but missing in the table. We are going to try to find another one in the table that contains this information, but perhaps this creates errors!
The column  Nachname_Autor  is mandatory but missing in the table. We are going to try to find another one in the table that contains this information, but perhaps this creates errors!
The column  Erscheinungsjahr  is mandatory but missing in the table. We are going to try to find another one in the table that contains this information, but perhaps this creates errors!


In [9]:
books_dataframe = dataframe.loc[dataframe["itemType"] == "book"]

In [10]:
books_dataframe

,key,itemType,Titel,bookTitle,firstName,lastName,publisher,place,date,language,numPages,ISBN,url
0,9QXAD3VR,book,Gattungspoetik in quantitativer Sicht. Das Wer...,NaN,Hanno,Ehrlicher,Digital Humanities Cooperation,,2020,deutsch,1-29,,https://www.digitalhumanitiescooperation.de/wp...
1,WWI35ZEX,book,Theorien von Autorschaft und Stil in Bewegung:...,NaN,Nanette,Rißler-Pipka,,,2019,German,,9783954770991,http://romanischestudien.de/index.php/rst/issu...
2,EH82H9XH,book,"Schäferei, Computer, Internet. Digital Humanit...",NaN,Reinhard,Krüger,V&R unipress,Göttingen/Bonn,2016,,,,
4,LYJ9N5F8,book,Multilingual corpora and multilingual corpus a...,NaN,Thomas,Schmidt,Benjamins,"Amsterdam, Philadelphia",2012,en,,,https://benjamins.com/catalog/hsm.14
8,4YGT9GT7,book,Literaturwissenschaft im digitalen Medienwandel,NaN,Christof,Schöch,Philologie im Netz,,2014,de,,,http://web.fu-berlin.de/phin/beiheft7/b7i.htm
13,I5T7SJZH,book,Internet und digitale Medien in der Romanistik...,NaN,Thomas,Stöber,Philologie im Netz,,2004,,,,http://web.fu-berlin.de/phin/beiheft2/b2i.htm


In [11]:
books_dataframe.drop(columns=["bookTitle"], inplace=True)

C:\Users\calvotello\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
chapters_dataframe = dataframe.loc[dataframe["itemType"] == "bookSection"]

In [13]:
chapters_dataframe

,key,itemType,Titel,bookTitle,firstName,lastName,publisher,place,date,language,numPages,ISBN,url
3,QEI4T8ZU,bookSection,Das aleatorische Netz des Wissens: Die Traditi...,Europäische Verlage und romanische Gegenwartsl...,Reinhard,Krüger,Narr,Tübingen,2003,de,NaN,,
5,3ZT9YJ33,bookSection,"Edición de textos, hipertextos y lexicografía",Actes du XXVe Congrès International de Linguis...,Harald,Völker,de Gruyter,Innsbruck,2010,,NaN,,https://doi.org/10.5167/uzh-45415
6,JTIY67LY,bookSection,A survey of spoken language corpora in Romance,Romanistische Korpuslinguistik. Korpora und ge...,Claus D.,Pusch,Narr,Tübingen,2003,de,NaN,,https://home.uni-leipzig.de/burr/CorpusLing/pd...
7,M6ED3TMH,bookSection,"Corneille, Molière et les autres. Stilometrisc...",Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,NaN,,http://web.fu-berlin.de/phin/beiheft7/b7t08.pdf
9,LZZQJCY7,bookSection,El estudio de revistas culturales en la era de...,Tramas impresas. Publicaciones periódicas arge...,Hanno,Ehrlicher,Universidad Nacional de La Plata: Facultad de ...,,2015,es,NaN,,
10,HXZMP8ZR,bookSection,Die Visualisierung von Varianten in der Texted...,Variante und Varietät: Akten des VI. Dies Roma...,Christof,Schöch,Edizioni ETS,Pisa,2013,de,NaN,,https://hal.archives-ouvertes.fr/hal-00945358
11,LBG6TH5T,bookSection,Avellaneda y los problemas de la identificació...,El otro Quijote. La continuación de Avellaneda...,Nanette,Rißler-Pipka,Universität Augsburg,Augsburg,2016,es,NaN,,https://opus.bibliothek.uni-augsburg.de/opus4/...
12,IV9ENEX6,bookSection,Sobre los problemas de investigación con revis...,Almacenes de un tiempo en fuga: Revistas cultu...,Nanette,Rißler-Pipka,,,2014,es,NaN,,https://www.revistas-culturales.de/de/buchseit...


In [14]:
chapters_dataframe["Titel"] = chapters_dataframe["bookTitle"]

C:\Users\calvotello\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
chapters_dataframe.drop(columns=["bookTitle"], inplace=True)

C:\Users\calvotello\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
chapters_dataframe

,key,itemType,Titel,firstName,lastName,publisher,place,date,language,numPages,ISBN,url
3,QEI4T8ZU,bookSection,Europäische Verlage und romanische Gegenwartsl...,Reinhard,Krüger,Narr,Tübingen,2003,de,NaN,,
5,3ZT9YJ33,bookSection,Actes du XXVe Congrès International de Linguis...,Harald,Völker,de Gruyter,Innsbruck,2010,,NaN,,https://doi.org/10.5167/uzh-45415
6,JTIY67LY,bookSection,Romanistische Korpuslinguistik. Korpora und ge...,Claus D.,Pusch,Narr,Tübingen,2003,de,NaN,,https://home.uni-leipzig.de/burr/CorpusLing/pd...
7,M6ED3TMH,bookSection,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,NaN,,http://web.fu-berlin.de/phin/beiheft7/b7t08.pdf
9,LZZQJCY7,bookSection,Tramas impresas. Publicaciones periódicas arge...,Hanno,Ehrlicher,Universidad Nacional de La Plata: Facultad de ...,,2015,es,NaN,,
10,HXZMP8ZR,bookSection,Variante und Varietät: Akten des VI. Dies Roma...,Christof,Schöch,Edizioni ETS,Pisa,2013,de,NaN,,https://hal.archives-ouvertes.fr/hal-00945358
11,LBG6TH5T,bookSection,El otro Quijote. La continuación de Avellaneda...,Nanette,Rißler-Pipka,Universität Augsburg,Augsburg,2016,es,NaN,,https://opus.bibliothek.uni-augsburg.de/opus4/...
12,IV9ENEX6,bookSection,Almacenes de un tiempo en fuga: Revistas cultu...,Nanette,Rißler-Pipka,,,2014,es,NaN,,https://www.revistas-culturales.de/de/buchseit...


In [17]:
dataframe = pd.concat([books_dataframe, chapters_dataframe])


In [18]:
dataframe.Titel = dataframe.Titel.str.findall(r"^(.*?)(?:$|[\.:])",  flags=re.I).str[0]

In [19]:
dataframe

,key,itemType,Titel,firstName,lastName,publisher,place,date,language,numPages,ISBN,url
0,9QXAD3VR,book,Gattungspoetik in quantitativer Sicht,Hanno,Ehrlicher,Digital Humanities Cooperation,,2020,deutsch,1-29,,https://www.digitalhumanitiescooperation.de/wp...
1,WWI35ZEX,book,Theorien von Autorschaft und Stil in Bewegung,Nanette,Rißler-Pipka,,,2019,German,,9783954770991,http://romanischestudien.de/index.php/rst/issu...
2,EH82H9XH,book,"Schäferei, Computer, Internet",Reinhard,Krüger,V&R unipress,Göttingen/Bonn,2016,,,,
4,LYJ9N5F8,book,Multilingual corpora and multilingual corpus a...,Thomas,Schmidt,Benjamins,"Amsterdam, Philadelphia",2012,en,,,https://benjamins.com/catalog/hsm.14
8,4YGT9GT7,book,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,,,http://web.fu-berlin.de/phin/beiheft7/b7i.htm
13,I5T7SJZH,book,Internet und digitale Medien in der Romanistik,Thomas,Stöber,Philologie im Netz,,2004,,,,http://web.fu-berlin.de/phin/beiheft2/b2i.htm
3,QEI4T8ZU,bookSection,Europäische Verlage und romanische Gegenwartsl...,Reinhard,Krüger,Narr,Tübingen,2003,de,NaN,,
5,3ZT9YJ33,bookSection,Actes du XXVe Congrès International de Linguis...,Harald,Völker,de Gruyter,Innsbruck,2010,,NaN,,https://doi.org/10.5167/uzh-45415
6,JTIY67LY,bookSection,Romanistische Korpuslinguistik,Claus D.,Pusch,Narr,Tübingen,2003,de,NaN,,https://home.uni-leipzig.de/burr/CorpusLing/pd...
7,M6ED3TMH,bookSection,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,NaN,,http://web.fu-berlin.de/phin/beiheft7/b7t08.pdf


# Extract Data from Catalogue

In [20]:
dataframe = extract_data.extract_fields_with_title(dataframe)

In [21]:
dataframe

,key,itemType,Titel,firstName,lastName,publisher,place,date,language,numPages,ISBN,url,nach_Titel_ppn,nach_Titel_medium,nach_Titel_Bestand_K10
0,9QXAD3VR,book,Gattungspoetik in quantitativer Sicht,Hanno,Ehrlicher,Digital Humanities Cooperation,,2020,deutsch,1-29,,https://www.digitalhumanitiescooperation.de/wp...,NaN,NaN,NaN
1,WWI35ZEX,book,Theorien von Autorschaft und Stil in Bewegung,Nanette,Rißler-Pipka,,,2019,German,,9783954770991,http://romanischestudien.de/index.php/rst/issu...,1668473194,Aau,12.0
2,EH82H9XH,book,"Schäferei, Computer, Internet",Reinhard,Krüger,V&R unipress,Göttingen/Bonn,2016,,,,,1011982072|886260493|870083570,Oan|Oau|Aau,30.0
4,LYJ9N5F8,book,Multilingual corpora and multilingual corpus a...,Thomas,Schmidt,Benjamins,"Amsterdam, Philadelphia",2012,en,,,https://benjamins.com/catalog/hsm.14,1683332830|1658845552|1651949999|81009858X|725...,Oau|Oan|Oau|Oan|Oax|Aau|Oan|Oax,54.0
8,4YGT9GT7,book,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,,,http://web.fu-berlin.de/phin/beiheft7/b7i.htm,NaN,NaN,NaN
13,I5T7SJZH,book,Internet und digitale Medien in der Romanistik,Thomas,Stöber,Philologie im Netz,,2004,,,,http://web.fu-berlin.de/phin/beiheft2/b2i.htm,NaN,NaN,NaN
3,QEI4T8ZU,bookSection,Europäische Verlage und romanische Gegenwartsl...,Reinhard,Krüger,Narr,Tübingen,2003,de,NaN,,,351929002,Aau,19.0
5,3ZT9YJ33,bookSection,Actes du XXVe Congrès International de Linguis...,Harald,Völker,de Gruyter,Innsbruck,2010,,NaN,,https://doi.org/10.5167/uzh-45415,1652159002|140095651X|1400956064|1400955815|14...,Oau|Afu|Afu|Afu|Afu|Afu|Afu|Afu|Oax|Oan|AFu|AF...,98.0
6,JTIY67LY,bookSection,Romanistische Korpuslinguistik,Claus D.,Pusch,Narr,Tübingen,2003,de,NaN,,https://home.uni-leipzig.de/burr/CorpusLing/pd...,1185754334|477280366|1168928176|495747572|3429...,Afu|AFu|Afu|Acu|AFu|Acu,58.0
7,M6ED3TMH,bookSection,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,NaN,,http://web.fu-berlin.de/phin/beiheft7/b7t08.pdf,NaN,NaN,NaN


# Check with SUB Catalogue

In [22]:
dataframe = check_duplicates.check_duplicate_with_title(dataframe)

In [23]:
dataframe

,key,itemType,Titel,firstName,lastName,publisher,place,date,language,numPages,ISBN,url,nach_Titel_ppn,nach_Titel_medium,nach_Titel_Bestand_K10,nach_Titel_Bestand_Göttingen,nach_Titel_Ort_Göttingen,nach_Titel_Medium_Göttingen,nach_Titel_URL_GUK
0,9QXAD3VR,book,Gattungspoetik in quantitativer Sicht,Hanno,Ehrlicher,Digital Humanities Cooperation,,2020,deutsch,1-29,,https://www.digitalhumanitiescooperation.de/wp...,NaN,NaN,NaN,0,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
1,WWI35ZEX,book,Theorien von Autorschaft und Stil in Bewegung,Nanette,Rißler-Pipka,,,2019,German,,9783954770991,http://romanischestudien.de/index.php/rst/issu...,1668473194,Aau,12.0,1,LS1,Aau,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
2,EH82H9XH,book,"Schäferei, Computer, Internet",Reinhard,Krüger,V&R unipress,Göttingen/Bonn,2016,,,,,1011982072|886260493|870083570,Oan|Oau|Aau,30.0,2,FMAG,Aau|Oau,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
4,LYJ9N5F8,book,Multilingual corpora and multilingual corpus a...,Thomas,Schmidt,Benjamins,"Amsterdam, Philadelphia",2012,en,,,https://benjamins.com/catalog/hsm.14,1683332830|1658845552|1651949999|81009858X|725...,Oau|Oan|Oau|Oan|Oax|Aau|Oan|Oax,54.0,2,7/052,Aau|Oau,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
8,4YGT9GT7,book,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,,,http://web.fu-berlin.de/phin/beiheft7/b7i.htm,NaN,NaN,NaN,0,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
13,I5T7SJZH,book,Internet und digitale Medien in der Romanistik,Thomas,Stöber,Philologie im Netz,,2004,,,,http://web.fu-berlin.de/phin/beiheft2/b2i.htm,NaN,NaN,NaN,0,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
3,QEI4T8ZU,bookSection,Europäische Verlage und romanische Gegenwartsl...,Reinhard,Krüger,Narr,Tübingen,2003,de,NaN,,,351929002,Aau,19.0,1,BBK-ROM|FMAG,Aau,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
5,3ZT9YJ33,bookSection,Actes du XXVe Congrès International de Linguis...,Harald,Völker,de Gruyter,Innsbruck,2010,,NaN,,https://doi.org/10.5167/uzh-45415,1652159002|140095651X|1400956064|1400955815|14...,Oau|Afu|Afu|Afu|Afu|Afu|Afu|Afu|Oax|Oan|AFu|AF...,98.0,2,,Acu|AFu,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
6,JTIY67LY,bookSection,Romanistische Korpuslinguistik,Claus D.,Pusch,Narr,Tübingen,2003,de,NaN,,https://home.uni-leipzig.de/burr/CorpusLing/pd...,1185754334|477280366|1168928176|495747572|3429...,Afu|AFu|Afu|Acu|AFu|Acu,58.0,3,BBK-ROM|BBK-MAG,Acu|AFu,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
7,M6ED3TMH,bookSection,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,NaN,,http://web.fu-berlin.de/phin/beiheft7/b7t08.pdf,NaN,NaN,NaN,0,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...


In [24]:
dataframe = check_duplicates.check_duplicate_with_title_author(dataframe, name_column_author="lastName")

In [25]:
dataframe

,key,itemType,Titel,firstName,lastName,publisher,place,date,language,numPages,...,nach_Titel_Bestand_K10,nach_Titel_Bestand_Göttingen,nach_Titel_Ort_Göttingen,nach_Titel_Medium_Göttingen,nach_Titel_URL_GUK,nach_Titel_lastName_Bestand_Göttingen,nach_Titel_lastName_Ort_Göttingen,nach_Titel_lastName_Medium_Göttingen,nach_Titel_lastName_year_Göttingen,nach_Titel_lastName_URL_GUK
0,9QXAD3VR,book,Gattungspoetik in quantitativer Sicht,Hanno,Ehrlicher,Digital Humanities Cooperation,,2020,deutsch,1-29,...,NaN,0,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,0,,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
1,WWI35ZEX,book,Theorien von Autorschaft und Stil in Bewegung,Nanette,Rißler-Pipka,,,2019,German,,...,12.0,1,LS1,Aau,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,1,LS1,Aau,2019,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
2,EH82H9XH,book,"Schäferei, Computer, Internet",Reinhard,Krüger,V&R unipress,Göttingen/Bonn,2016,,,...,30.0,2,FMAG,Aau|Oau,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,2,FMAG,Aau|Oau,2016,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
4,LYJ9N5F8,book,Multilingual corpora and multilingual corpus a...,Thomas,Schmidt,Benjamins,"Amsterdam, Philadelphia",2012,en,,...,54.0,2,7/052,Aau|Oau,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,2,7/052,Aau|Oau,2012,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
8,4YGT9GT7,book,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,,...,NaN,0,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,0,,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
13,I5T7SJZH,book,Internet und digitale Medien in der Romanistik,Thomas,Stöber,Philologie im Netz,,2004,,,...,NaN,0,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,0,,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
3,QEI4T8ZU,bookSection,Europäische Verlage und romanische Gegenwartsl...,Reinhard,Krüger,Narr,Tübingen,2003,de,NaN,...,19.0,1,BBK-ROM|FMAG,Aau,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,0,,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
5,3ZT9YJ33,bookSection,Actes du XXVe Congrès International de Linguis...,Harald,Völker,de Gruyter,Innsbruck,2010,,NaN,...,98.0,2,,Acu|AFu,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,0,,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
6,JTIY67LY,bookSection,Romanistische Korpuslinguistik,Claus D.,Pusch,Narr,Tübingen,2003,de,NaN,...,58.0,3,BBK-ROM|BBK-MAG,Acu|AFu,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,3,BBK-ROM|BBK-MAG,Acu|AFu,2002|2005,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...
7,M6ED3TMH,bookSection,Literaturwissenschaft im digitalen Medienwandel,Christof,Schöch,Philologie im Netz,,2014,de,NaN,...,NaN,0,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...,0,,,,https://opac.sub.uni-goettingen.de/DB=1/SET=2/...


In [26]:
dataframe.to_csv("./../data/zotero_bibliography_controlled.tsv", sep="\t")